In [1]:
from datasets import load_dataset
from experiments.dataset_info import *

In [2]:
amazon_dict

{'All_Beauty': 0,
 'Toys_and_Games': 1,
 'Cell_Phones_and_Accessories': 2,
 'Industrial_and_Scientific': 3,
 'Gift_Cards': 4,
 'Musical_Instruments': 5,
 'Electronics': 6,
 'Handmade_Products': 7,
 'Arts_Crafts_and_Sewing': 8,
 'Baby_Products': 9,
 'Health_and_Household': 10,
 'Office_Products': 11,
 'Digital_Music': 12,
 'Grocery_and_Gourmet_Food': 13,
 'Sports_and_Outdoors': 14,
 'Home_and_Kitchen': 15,
 'Subscription_Boxes': 16,
 'Tools_and_Home_Improvement': 17,
 'Pet_Supplies': 18,
 'Video_Games': 19,
 'Kindle_Store': 20,
 'Clothing_Shoes_and_Jewelry': 21,
 'Patio_Lawn_and_Garden': 22,
 'Unknown': 23,
 'Books': 24,
 'Automotive': 25,
 'CDs_and_Vinyl': 26,
 'Beauty_and_Personal_Care': 27,
 'Amazon_Fashion': 28,
 'Magazine_Subscriptions': 29,
 'Software': 30,
 'Health_and_Personal_Care': 31,
 'Appliances': 32,
 'Movies_and_TV': 33}

In [3]:
# Inspect different datasets within amazon_reviews
# dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_All_Beauty", trust_remote_code=True)
# dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_All_Beauty", trust_remote_code=True)
# dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "0core_rating_only_All_Beauty", trust_remote_code=True)
# dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "0core_last_out_All_Beauty", trust_remote_code=True)
# dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "0core_timestamp_w_his_All_Beauty", trust_remote_code=True)

In [4]:
dataset_beauty = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_All_Beauty", trust_remote_code=True, streaming=True)
ds_beauty = iter(dataset_beauty["full"])


In [5]:
next(ds_beauty)

{'rating': 5.0,
 'title': 'Such a lovely scent but not overpowering.',
 'text': "This spray is really nice. It smells really good, goes on really fine, and does the trick. I will say it feels like you need a lot of it though to get the texture I want. I have a lot of hair, medium thickness. I am comparing to other brands with yucky chemicals so I'm gonna stick with this. Try it!",
 'images': [],
 'asin': 'B00YQ6X8EO',
 'parent_asin': 'B00YQ6X8EO',
 'user_id': 'AGKHLEW2SOWHNMFQIJGBECAF7INQ',
 'timestamp': 1588687728923,
 'helpful_vote': 0,
 'verified_purchase': True}

In [6]:
dataset_electronics = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_Electronics", trust_remote_code=True, streaming=True)
ds_electronics = iter(dataset_electronics["full"])

In [7]:
next(ds_electronics)

{'rating': 3.0,
 'title': 'Smells like gasoline! Going back!',
 'text': 'First & most offensive: they reek of gasoline so if you are sensitive/allergic to petroleum products like I am you will want to pass on these.  Second: the phone adapter is useless as-is. Mine was not drilled far enough to be able to tighten it into place for my iPhone 12 max. It just slipped & slid all over. Stupid me putting the adapter together first without picking up the binoculars to smell them bc I wasted 15 minutes trying to figure out how to put the adapter together bc it does not come with instructions!  I had to come back here to the website which was a total pain. Third: the tripod is also useless. I would not trust the iOS to hold my $1600 phone nor even a Mattel Barbie for that matter. It’s just inefficient for the job imo.  Third: in order to try to give an honest review I did don gloves & eyewear to check the binoculars out.  They seemed average except for mine seemed to be missing about 10% of the

In [8]:
dataset_electronics = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_Electronics", trust_remote_code=True, streaming=True)


# Load balanced dataset of 10_000 samples for each 0.0 / 5.0 rating vs All_beauty / Electronics
# three columns per sample: f'Title: {title}\nReview: {review}'[:1000], rating, category

# write code using dataset.select()


In [10]:

from datasets import load_dataset, Dataset, DatasetDict
from collections import defaultdict
import random


def process_sample(sample, category):
    return {
        'text': f'Title: {sample["title"]}\nReview: {sample["text"]}'[:1000],
        'rating': sample['rating'],
        'category': category,
    }

def filter_and_sample(dataset, ratings, category, n_samples):
    rating_counter = {rating: 0 for rating in ratings}
    samples_dict = defaultdict(list)
    for sample in dataset:
        # print(rating_counter)
        r = sample['rating'] 
        if r in rating_counter:
            samples_dict[r].append(process_sample(sample, category))
            rating_counter[r] += 1
            if rating_counter[r] >= n_samples:
                rating_counter.pop(r) # desired number of samples reached
        if len(rating_counter) == 0:
            break
    return samples_dict

def split_samples_dict(samples_dict, n_train_samples_per_rating, n_test_samples_per_rating):
    train_samples = []
    test_samples = []
    for samples in samples_dict.values():
        train_samples.extend(samples[:n_train_samples_per_rating])
        test_samples.extend(samples[n_train_samples_per_rating:n_train_samples_per_rating+n_test_samples_per_rating])
    return train_samples, test_samples # contains multiple rating_scores


# Load the datasets
dataset_electronics = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_Electronics", trust_remote_code=True, streaming=True)
dataset_beauty = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_All_Beauty", trust_remote_code=True, streaming=True)
dataset_dict = {
    'electronics': dataset_electronics['full'],
    'beauty': dataset_beauty['full'],
}
ratings = (1.0, 5.0) # There's no 0 star rating in the dataset
n_train_samples_per_rating = 25000
n_test_samples_per_rating = 10000
n_total_samples_per_rating = n_train_samples_per_rating + n_test_samples_per_rating

train_samples = []
test_samples = []
for category, dataset in dataset_dict.items():
    print(f"Processing {category} dataset...")
    samples_dict = filter_and_sample(dataset, ratings, category, n_total_samples_per_rating)
    train_samples_category, test_samples_category = split_samples_dict(samples_dict, n_train_samples_per_rating, n_test_samples_per_rating)
    train_samples.extend(train_samples_category)
    test_samples.extend(test_samples_category)

balanced_dataset = DatasetDict({
    'train': Dataset.from_list(train_samples),
    'test': Dataset.from_list(test_samples),
})

print(len(balanced_dataset['train']), len(balanced_dataset['test']))

Processing electronics dataset...
Processing beauty dataset...
100000 40000


In [13]:
# Upload to huggingface

# ratings_part = "_".join(str(rating) for rating in ratings)
# categories_part = "_".join(dataset_dict.keys())

# fname = f"dataset_{ratings_part}_{categories_part}_{n_total_samples_per_rating}"

# balanced_dataset.push_to_hub(repo_id = f"canrager/amazon_reviews_mcauley", data_dir=f"{fname}")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/100 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/40 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/canrager/amazon_reviews_mcauley/commit/7686472154fe779356485f175567c4de1268bdde', commit_message='Upload dataset', commit_description='', oid='7686472154fe779356485f175567c4de1268bdde', pr_url=None, pr_revision=None, pr_num=None)

### Inspect how bib is formatted

In [10]:
from experiments.probe_training import load_and_prepare_dataset


bib_train_df, bib_test_df = load_and_prepare_dataset('bias_in_bios')

In [12]:
bib_train_df.head()

,hard_text,profession,gender
0,"He specializes in development economics, house...",21,0
1,He started out as a DJ and music producer in t...,5,0
2,"She is averse to all things scary or sad, so s...",4,1
3,"Prior to joining USC, she was a mobile news ed...",21,1
4,"Previously, she served as an assistant profess...",21,1


In [13]:
len(bib_train_df), len(bib_test_df)

(257478, 99069)

In [ ]:
# Try running stuff with this dataset: replicate test_interventions.py